# Logic gates sequence finder
### Genetic Algorithms
In this computer assignment we use genetic algorithm to find solution to our gate problem.  
Genetic algorithms are used in problems with large problem space.  
This algorithm is based on natural selection, it has an initial population which then evolve  
using crossover and mutation.  
Also in each cycle we select the new population from the previous one.  


In [ ]:
from typing import List
from random import randint, uniform, shuffle, sample
import enum
import time
import pandas as pd
import numpy as np

In [ ]:
truth_table_df = pd.read_csv("truth_table.csv")
CIRCUIT_LENGHT = truth_table_df.shape[1] - 2
ROW_COUNT = truth_table_df.shape[0]

### Hyperparameters
An important part in every data related project is finding the right parameters for our data.  
I have used 4 hyperparameters, INITIAL_POPULATION, P_C, P_M, RECURRING_FITNESS_LIMIT.  

INITIAL_POPULATION is equal to 2^(CIRCUIT_LENGHT - 1) because when we have more inputs  
It is more complicated and we need more individuals in our population.

P_C is 0.55 and P_M is 1/(CIRCUIT_LENGTH + 1), I reached to this parameters by lots of different choices that I have mentioned here.

RECURRING_FITNESS_LIMIT is used to change the population when it is repeating itself. Also I have reached this parameter by choosing different choices.

RECURRING_P_C is used to determine what is used for P_C in the recurring situation

#### My Hyperparameter choices history  
some data may not be included  
INITIAL_POPULATION: 200 => 2^(CIRCUIT_LENGTH - 1)  
P_C: 0.7 => 0.3 => 0.1 => 0.7 => 0.5 => 0.6 => 0.45 => 0.7 => 0.55  
P_M: 0.005 => 0.1 => 0.2 => 0.01 => 0.005 => 0.1  => 1/(CIRCUIT_LENGTH + 1)  
RECURRING_FITNESS_LIMIT: 7 => 14 => 20 => 50 => 35 => 30 => 10  
RECURRING_P_C: 0.4

In [ ]:
# hyperparameters
INITIAL_POPULATION = 2**(CIRCUIT_LENGHT - 1)
P_C = 0.55
P_M = 1 / (CIRCUIT_LENGHT + 1)
RECURRING_FITNESS_LIMIT = 10
RECURRING_P_C = 0.4

### Gene
we choose Gates as our genes in our problem  
Each gate can have a value from 0 to 5 that correspons to its Logic gate.

### Chromosome
Each Chromosome is a list of gates(genes) and its ordered.  
Circuit class is used to implement the Chromosome.

In [ ]:
class Gate(enum.IntEnum):
    AND = 0
    OR = 1
    XOR = 2
    NAND = 3
    NOR = 4
    XNOR = 5
    
    def __get_and_output(self, input1, input2):
        return input1 and input2

    def __get_or_output(self, input1, input2):
        return input1 or input2

    def __get_xor_output(self, input1, input2):
        return input1 != input2

    def __get_nand_output(self, input1, input2):
        return not (input1 and input2)

    def __get_nor_output(self, input1, input2):
        return not (input1 or input2)

    def __get_xnor_output(self, input1, input2):
        return input1 == input2

    def get_output(self, input1, input2):
        if self.value == 0:
            return self.__get_and_output(input1, input2)
        if self.value == 1:
            return self.__get_or_output(input1, input2)
        if self.value == 2:
            return self.__get_xor_output(input1, input2)
        if self.value == 3:
            return self.__get_nand_output(input1, input2)
        if self.value == 4:
            return self.__get_nor_output(input1, input2)
        if self.value == 5:
            return self.__get_xnor_output(input1, input2)

### Circuit class:

#### Fitness:
I implemented a get_fitness mehtod within this class to get the fitness.  
my fitness is defined as count of equal answers of my circuit and the dataset outputs.  
This is a good fitness because it can determine how good we are doing.

#### Crossover:
this method gets another Circuit and does 1-point crossover between them  
We could have done n-point or uniform crossover too  

#### Mutate:
we change the Gate number to 5 - Gate number if the probability condition is true.  
This is done on every gene(Gate) of the Chromosome(Circuit).

In [ ]:
class Circuit:
    def __init__(self, gates: List[Gate]):
        self.gates = gates

    def get_output(self, inputs: List[bool]):
        assert len(inputs) >= 2, "inputs length must be greater than 1"
        output = self.gates[0].get_output(inputs[0], inputs[1])
        for i in range(2, len(inputs)):
            output = self.gates[i - 1].get_output(output, inputs[i])
        return output
    
    def get_fitness(self, truth_table_df):
        similarity = truth_table_df.iloc[:, :-1].apply(lambda row: self.get_output(row.values.tolist()), axis=1) == truth_table_df.iloc[:, -1]
        return np.count_nonzero(similarity)
    
    def crossover(self, other):
            random = randint(0, len(self.gates))
            new_self_gates = self.gates[0:random] + other.gates[random:]
            new_other_gates = other.gates[0:random] + self.gates[random:]
            return Circuit(new_self_gates), Circuit(new_other_gates)

    def mutate(self, probability):
        changed = False
        p = uniform(0, 1)
        for i in range(len(self.gates)):
            if (p < probability):
                changed = True
                self.gates[i] = Gate(5 - self.gates[i].value)
        return changed

### CircuitsContainer:
This class is used to manage our population of circuits in the program.  
For time order issues we also keep a dictionary of Circuits and fitnesses so that we do not  
calculate them twice or more.  

#### Select:
We use Tournoment select for this part.  
It chooses k circuits from the population and then we compare the fitnesses.  
The circuit that has the biggest fitness wins and can remain in our population.  
I used k=2 in my implementation.  

#### Crossover:
It shuffles the circuits and crossovers them two by two.

#### Mutation:
In this part we call the mutation method for each circuit.  
This part does another important thing, when we reach our recurring limit  
it changes the p_m for one generation so that the population can change more and then it goes back to its intital value.

In [ ]:
class CircuitsContainer:
    def __init__(self):
        self.circuits = []
        self.circuits_dict = {}
        self.generation_max_fitness_history = []

    def add_to_dict(self, circuits):
        for circuit in circuits:
            if circuit not in self.circuits_dict:
                self.circuits_dict[circuit] = circuit.get_fitness(truth_table_df)

    def add_circuit(self, gates: List[int]):
        c = Circuit([Gate(i) for i in gates])
        self.circuits.append(c)
        self.add_to_dict([c])
    
    def select(self):
        shuffle(self.circuits)
        new_circuits = []
        i = 0
        while i < len(self.circuits) - 1:
            self.add_to_dict([self.circuits[i], self.circuits[i+1]])
            if self.circuits_dict[self.circuits[i]] > self.circuits_dict[self.circuits[i+1]]:
                new_circuits.append(self.circuits[i])
            else:
                new_circuits.append(self.circuits[i+1])
            i += 2
        shuffle(self.circuits)
        i = 0
        while i < len(self.circuits) - 1:
            self.add_to_dict([self.circuits[i], self.circuits[i+1]])
            if self.circuits_dict[self.circuits[i]] > self.circuits_dict[self.circuits[i+1]]:
                new_circuits.append(self.circuits[i])
            else:
                new_circuits.append(self.circuits[i+1])
            i += 2
        self.circuits = new_circuits

    def crossover(self):
        shuffle(self.circuits)
        new_circuits = []
        i = 0
        while i < len(self.circuits) - 1:
            if (uniform(0, 1) < P_C):
                new_circuit1, new_circuit2 = self.circuits[i].crossover(self.circuits[i+1])
                new_circuits.extend([new_circuit1, new_circuit2])
                self.add_to_dict([new_circuit1, new_circuit2])
            else:
                new_circuits.extend([self.circuits[i], self.circuits[i+1]])
            i += 2
        self.circuits = new_circuits
    
    def mutate(self):
        probability = P_M
        if len(self.generation_max_fitness_history) > 0:
            if self.generation_max_fitness_history.count(self.generation_max_fitness_history[-1]) > RECURRING_FITNESS_LIMIT:        
                print("*********************************************")
                print("MAXIMUM RECURRING LIMIT REACHED!")
                print("CHANGING P_M for this step")
                print("*********************************************")
                self.generation_max_fitness_history = []
                probability = RECURRING_P_C
        counter = 0
        for circuit in self.circuits:
            has_changed = circuit.mutate(probability)
            if (has_changed):
                counter += 1
                self.circuits_dict[circuit] = circuit.get_fitness(truth_table_df)

    def check_answer(self):
        fitness_max = 0
        for circuit, fitness in self.circuits_dict.items():
            if fitness > fitness_max:
                fitness_max = fitness
            if fitness == ROW_COUNT:
                return True, circuit
        print("GENERATION MAX FITNESS:")
        print(fitness_max)
        self.generation_max_fitness_history.append(fitness_max)
        return False, None

### Initial Population
In this step we created the initial population.  
the initial population is created by using a numpy random ndarray.  
Its population is determined by the hyperparameter INITIAL_POPULATION

In [ ]:
circuits_container = CircuitsContainer()
random = pd.DataFrame(np.random.randint(0, 6, size=(INITIAL_POPULATION, CIRCUIT_LENGHT)))
a = random.apply(lambda row: circuits_container.add_circuit(row.values.tolist()), axis=1)

#### Running the algorithm
We do the following steps in order:
1. selecting from the population
2. crossover
3. mutation
4. check if we have reached the answer or not

these steps are repated until an answer is found

In [ ]:
tic = time.time()
answer = None
counter = 1
while True:
    print("****************************************")
    print("GENERATION NUMBER:")
    print(counter)
    circuits_container.select()
    circuits_container.crossover()
    circuits_container.mutate()
    is_answer, answer = circuits_container.check_answer()
    if (is_answer):
        print("FOUND THE ANSWER!")
        break
    counter += 1
toc = time.time()

In [ ]:
print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print("GENERATIONS CREATED: " + str(counter))
print("Time taken: " + str(toc - tic))
print("ANSWER: ", end="")
for gate in answer.gates:
 print(gate, end="    ")
print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

#### QUESTIONS
1. we used this fitness because it shows how far we are from 1024(conditions) and we can determine when we reached our goal


2. we used tournoment selection, further explaines are available in selection part.


3. mutation causes each genes to be changed a little individually by a probabilty.  
    and crossover helps the chromoseomes to get good parts of eachother and get better.


4. This is a probable issue, I store history of generations and if we are repeating on a fitness for example for 10 times
    Then i change the probabilty of p_m for one generation to get out of this situation.

